In [1]:
"""Compute normalization statistics for a config.

This script is used to compute the normalization statistics for a given config. It
will compute the mean and standard deviation of the data in the dataset and save it
to the config assets directory.
"""

import numpy as np
import tqdm
import tyro

import openpi.shared.normalize as normalize
import openpi.training.config as _config
import openpi.training.data_loader as _data_loader
import openpi.transforms as transforms


class RemoveStrings(transforms.DataTransformFn):

    def __call__(self, x: dict) -> dict:
        return {k: v for k, v in x.items() if not np.issubdtype(np.asarray(v).dtype, np.str_)}


def create_dataset(config: _config.TrainConfig, ) -> tuple[_config.DataConfig, _data_loader.Dataset]:
    data_config = config.data.create(config.assets_dirs, config.model)
    if data_config.repo_id is None:
        raise ValueError("Data config must have a repo_id")
    dataset = _data_loader.create_dataset(data_config, config.model)
    dataset = _data_loader.TransformedDataset(
        dataset,
        [
            *data_config.repack_transforms.inputs,
            *data_config.data_transforms.inputs,
            # Remove strings since they are not supported by JAX and are not needed to compute norm stats.
            RemoveStrings(),
        ],
    )
    return data_config, dataset

In [2]:
config_name="pi0_base_agibot_lora"
config = _config.get_config(config_name)
data_config, dataset = create_dataset(config)

num_frames = len(dataset)
shuffle = False
data_loader = _data_loader.TorchDataLoader(
        dataset,
        local_batch_size=8,
        num_workers=0,
        shuffle=shuffle,
        num_batches=num_frames,
    )

keys = ["state", "actions"]
stats = {key: normalize.RunningStats() for key in keys}
print(f"Computing normalization statistics for {num_frames} frames...")

Computing normalization statistics for 6032 frames...


In [3]:
for batch in tqdm.tqdm(data_loader, total=num_frames, desc="Computing stats"):
      print("111111111111")
#     # 输出一些batch的信息（比如他有哪些）
#     batch_add= preprocess(batch)
#     # 在batch中添加新的键值对batch_add
#     batch.update(batch_add)
#     for key in keys:
#         values = np.asarray(batch[key][0])
#         stats[key].update(values.reshape(-1, values.shape[-1]))

# norm_stats = {key: stats.get_statistics() for key, stats in stats.items()}

# output_path = config.assets_dirs / data_config.repo_id
# print(f"Writing stats to: {output_path}")
# normalize.save(output_path, norm_stats)

Computing stats:   0%|                                                                                                                                                       | 0/6032 [00:00<?, ?it/s]

KeyError: "Column action not in the dataset. Current columns in the dataset: ['observation.images.path', 'observation.states', 'actions', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']"